Inspiration: https://towardsdatascience.com/the-best-document-similarity-algorithm-in-2020-a-beginners-guide-a01b9ef8cf05

This notebook describes the dataset and gets useful insights from it:
- similarity between different courts
- similarity to other text corpora like Wikipedia, CS Arxiv or News Corpora, German Court Decisions (openlegaldata)

How to compute similarity with Sentence BERT:
1. Sentence BERT to encode each sentence => sentence embedding
2. Average sentence embeddings of a document => document embedding
3.
    a) pairwise similarity of documents (one from each corpus)
    b) average document embeddings => corpus embedding
    => cosine similarity between corpus embeddings

Disclaimer: These aggregations of embeddings do not work very well apparently

TODO publish these notebooks as medium posts!

In [47]:
!python -m spacy download de_core_news_md # download spacy model

     |████████████████████████████████| 49.6 MB 4.6 kB/s  eta 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_md')


In [48]:
from pathlib import Path

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)
import random
from somajo import SoMaJo # German tokenizer
tokenizer = SoMaJo("de_CMC")

import spacy
import de_core_news_md
# disable pipelines for faster processing since we only need the vectors
nlp = de_core_news_md.load(disable=['tagger', 'parser', 'ner', 'lemmatizer', 'textcat']) 

INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [43]:
# downloading german glove word embeddings provided by deepset: https://deepset.ai/german-word-embeddings
!wget https://int-emb-glove-de-wiki.s3.eu-central-1.amazonaws.com/vectors.txt

--2021-03-05 10:35:28--  https://int-emb-glove-de-wiki.s3.eu-central-1.amazonaws.com/vectors.txt
Resolving int-emb-glove-de-wiki.s3.eu-central-1.amazonaws.com (int-emb-glove-de-wiki.s3.eu-central-1.amazonaws.com)... 52.219.47.60
Connecting to int-emb-glove-de-wiki.s3.eu-central-1.amazonaws.com (int-emb-glove-de-wiki.s3.eu-central-1.amazonaws.com)|52.219.47.60|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3744610526 (3.5G) [text/plain]
Saving to: ‘vectors.txt’

vectors.txt         100%[===================>]   3.49G  22.7MB/s    in 2m 35s  

2021-03-05 10:38:04 (23.1 MB/s) - ‘vectors.txt’ saved [3744610526/3744610526]



In [44]:
# make blank pipeline
!python -m spacy init vectors de vectors.txt /tmp/de_vectors_deepset

ℹ Creating blank nlp object for language 'de'
Reading vectors from vectors.txt
1309281it [01:29, 14618.05it/s]
Loaded vectors from vectors.txt
✔ Successfully converted 1309281 vectors
✔ Saved nlp object with vectors to output directory. You can now use
the path to it in your config as the 'vectors' setting in [initialize].
/tmp/de_vectors_deepset


In [49]:
spacy_vectors = spacy.load('/tmp/de_vectors_deepset')

In [3]:
data_dir = Path('../../data')
csv_dir = data_dir / 'csv'
clean_csv_dir = csv_dir / 'clean'

In [61]:
court_dir = clean_csv_dir / '_de.csv'
court_dir

PosixPath('../../data/csv/clean/_de.csv')

In [ ]:
# select only small random subset for testing
# keep the header, then take only 1% of lines
# if random from [0,1] interval is greater than 0.01 the row will be skipped
p = 0.001
df = pd.read_csv(
             court_dir,
             header=0, 
             skiprows=lambda i: i>0 and random.random() > p
    )

In [ ]:
df.head()

In [ ]:
bger_df = df[df['chamber'] == '']

In [50]:
doc1 = spacy_vectors(df['text'].iloc[1])
doc2 = spacy_vectors(df['text'].iloc[2])

In [54]:
news_article = "Die UBS passt ihre Gewinnzahlen für das abgelaufene Geschäftsjahr 2020 geringfügig nach unten an. So beträgt der Jahresgewinn nun 6.557 Milliarden Franken, wie die Grossbank in einer Mitteilung zur Veröffentlichung des Geschäftsberichtes mitteilte. CEO Ralph Hamers ist seit September in der Bank. Er erhielt 4.2 Millionen Franken. Sein Vorgänger Sergio Ermotti verdiente 2020 13.3 Millionen Franken. Das Ergebnis liegt 72 Millionen Franken unter dem Ende Januar bei der Veröffentlichung des Ergebnisses des vierten Quartals genannten Betrags, wie die Grossbank in einer Mitteilung zum Geschäftsbericht schreibt. Ralph Hamers, der am 1. September 2020 bei der UBS begonnen und das Amt des CEO am 1. November übernommen hat, erhielt insgesamt 4.2 Millionen Franken (siehe Tabelle unten). Zusätzlich leistete die Bank dem neuen Konzernchef eine einmalige Ersatzzahlung von 0.16 Millionen Franken."
doc3 = spacy_vectors(news_article)

In [56]:
short_text = "Das Grundwasser ist aufgefüllt – dank des vielen Regens und Schnees. Das sind gute Aussichten für den Sommer."
doc4 = spacy_vectors(short_text)

In [64]:
twitter = "Zur Amtszeit von Angela Merkel zählen übrigens die Verzehnfachung der Tafeln und die Verdreifachung der Sozialhilfeempfänger. Nur falls jemand noch eine Liste ihrer Erfolge machen möchte."
doc5 = spacy_vectors(twitter)

In [65]:
doc1.similarity(doc5)

0.969398834238912

In [20]:
sents = tokenizer.tokenize_text([df['text'].iloc[0]])
sentences = [" ".join([token.text for token in sent]) for sent in sents]    
print(sentences)

['RechtsprechungGericht / Verwaltung :ObergerichtAbteilung: Schuldbetreibungs- und KonkurskommissionRechtsgebiet : Schuldbetreibungs- und KonkursrechtEntscheiddatum : 15. 01. 1996 Fallnummer :O G 1996 42LGVE : 1996 I Nr. 42Leitsatz : Art .', '80 Abs. 2 SchKG ; § 110 Abs. 1 lit. e VRG ; §§ 18 , 24 , 27 und 28 Perimeterverordnung ( SRL Nr. 732 ) .', 'Eine Verfügung für Perimeterbeiträge bildet keinen definitiven Rechtsöffnungstitel , wenn sie keine Unterschrift aufweist und nicht mit einer Rechtsmittelbelehrung versehen ist .', 'Rechtskraft : Diese Entscheidung ist rechtskräftig .', 'Entscheid : Die Klägerin ( Strassengenossenschaft X. ) erliess eine Verfügung für Perimeterbeiträge an den Beklagten .', 'Nachdem dieser die Forderung nicht bezahlte , liess ihn die Klägerin betreiben .', 'Der Amtsgerichtspräsident lehnte das Gesuch der Klägerin um definitive Rechtsöffnung ab .', 'Er war zum Schluss gekommen , der Beklagte sei in der Beitragsverfügung nicht auf sein Einspracherecht hingewies

In [ ]:
# Import library and load multilingual sts model
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('stsb-xlm-r-multilingual')

In [41]:
%%time
sentence_embeddings = model.encode(sentences, convert_to_tensor=False)
document_embedding = np.mean(sentence_embeddings, axis=0)
document_embedding

CPU times: user 52.2 s, sys: 415 ms, total: 52.6 s
Wall time: 4.4 s


array([ 3.62253599e-02,  3.62150788e-01,  6.24165475e-01,  3.52770723e-02,
        2.53154427e-01,  1.21447854e-01,  3.86505276e-01, -4.16817933e-01,
        6.59541562e-02, -2.25426808e-01, -1.03606731e-01,  5.64076364e-01,
       -2.71009207e-02, -3.53103247e-03, -1.96921423e-01,  1.90958932e-01,
        9.71556604e-02, -4.59400505e-01,  1.92318499e-01, -3.52056175e-01,
       -3.02016214e-02, -1.43347442e-01,  3.68704915e-01,  3.28014016e-01,
        1.80625141e-01,  8.37689415e-02,  1.72338054e-01,  2.09026456e-01,
       -3.33660960e-01,  2.81649947e-01,  3.38599771e-01, -3.64035040e-01,
       -4.38539863e-01, -2.67154843e-01,  2.20296010e-02,  3.23379636e-01,
        2.56965965e-01, -2.56803352e-02, -1.79340407e-01, -2.47767959e-02,
        1.34962857e-01,  2.16369197e-01, -3.83294262e-02, -1.06307762e-02,
       -9.76707116e-02,  1.28304467e-01,  7.86920190e-02,  7.51289725e-03,
       -3.60536367e-01,  4.38903004e-01, -3.74700986e-02,  1.32296041e-01,
       -1.75871029e-01,  

In [21]:
%%time
#Compute embedding for both lists
embeddings = model.encode(sentences, convert_to_tensor=True)
embeddings

CPU times: user 55.6 s, sys: 619 ms, total: 56.2 s
Wall time: 4.65 s


tensor([[-0.2182,  0.9161,  0.6241,  ..., -0.1870, -0.0754,  0.4477],
        [ 0.3327,  0.4051,  1.0339,  ...,  0.5209, -0.0911,  0.1146],
        [ 0.3648,  0.6245,  0.4467,  ...,  0.3426, -1.2798,  0.4257],
        ...,
        [-0.1287,  0.4577,  0.9480,  ...,  0.8484, -0.9659,  0.0572],
        [-0.3768,  0.6009,  0.2132,  ..., -0.2803, -0.6714,  0.0545],
        [-0.2823,  0.6130,  0.6167,  ..., -0.1095,  0.7224,  0.0582]])

In [22]:
%%time
#Compute cosine-similarities for each sentence with each other sentence
cosine_scores = util.pytorch_cos_sim(embeddings, embeddings)
cosine_scores

CPU times: user 5.25 ms, sys: 7.85 ms, total: 13.1 ms
Wall time: 3.37 ms


tensor([[1.0000, 0.6355, 0.3567, 0.3815, 0.6680, 0.1293, 0.1690, 0.2635, 0.6317,
         0.4324, 0.3361, 0.7097, 0.5242, 0.3436, 0.5183, 0.4017, 0.4791, 0.2450,
         0.3860, 0.3609, 0.3296, 0.1138, 0.4443, 0.6126],
        [0.6355, 1.0000, 0.2987, 0.4078, 0.5720, 0.0523, 0.0997, 0.1749, 0.4853,
         0.2504, 0.2321, 0.6739, 0.5604, 0.2704, 0.5278, 0.2966, 0.3724, 0.2294,
         0.3292, 0.4764, 0.2897, 0.1366, 0.4493, 0.6984],
        [0.3567, 0.2987, 1.0000, 0.3897, 0.5068, 0.3724, 0.3644, 0.6878, 0.3966,
         0.6686, 0.5446, 0.5497, 0.4858, 0.4481, 0.4125, 0.3645, 0.5129, 0.4895,
         0.5520, 0.3741, 0.5321, 0.5324, 0.3218, 0.0720],
        [0.3815, 0.4078, 0.3897, 1.0000, 0.5230, 0.2164, 0.3269, 0.3148, 0.4569,
         0.4455, 0.3475, 0.5181, 0.7885, 0.3613, 0.3974, 0.2851, 0.3668, 0.4197,
         0.3888, 0.6255, 0.2453, 0.4414, 0.7772, 0.2406],
        [0.6680, 0.5720, 0.5068, 0.5230, 1.0000, 0.3340, 0.2267, 0.3172, 0.5798,
         0.5189, 0.3802, 0.7760, 0.6042

In [37]:
#Find the pairs with the highest cosine similarity scores
pairs = []
for i in range(len(cosine_scores)-1):
    for j in range(i+1, len(cosine_scores)):
        pairs.append({'index': [i, j], 'score': cosine_scores[i][j]})

#Sort scores in decreasing order
pairs = sorted(pairs, key=lambda x: x['score'], reverse=False)


print(f"Number of pairs: {len(pairs)}")
print(f"Average score: {np.std([x['score'] for x in pairs])}")

for pair in pairs[0:10]:
    i, j = pair['index']
    print("\n")
    print(sentences[i])
    print(sentences[j])
    print(f"=> Score: {pair['score']:.4f}")
    print("\n")
    print("="*100)

Number of pairs: 276
Average score: 0.16329775750637054


Der Amtsgerichtspräsident lehnte das Gesuch der Klägerin um definitive Rechtsöffnung ab .
Der Verordnungsgeber hat im Zusammenhang mit den Beiträgen an Bau , Betriebs und Unterhaltskosten eines öffentlichen Werkes von Genossenschaften ausdrücklich auch die allgemeinen Verfahrensvorschriften ( u. a. Beitragsverfügung und Rechtsmittel : § 24 PV ) als anwendbar erklärt ( § 27 PV ) .
=> Score: 0.0051




Dass diese Mängel anderweitig behoben worden sind , wird von der Klägerin nicht dargetan und ist den Akten auch nicht zu entnehmen .
80 Abs. 2 SchKG verneint und das klägerische Gesuch abgewiesen .
=> Score: 0.0253




Nachdem dieser die Forderung nicht bezahlte , liess ihn die Klägerin betreiben .
80 Abs. 2 SchKG verneint und das klägerische Gesuch abgewiesen .
=> Score: 0.0513




80 Abs. 2 SchKG ; § 110 Abs. 1 lit. e VRG ; §§ 18 , 24 , 27 und 28 Perimeterverordnung ( SRL Nr. 732 ) .
Nachdem dieser die Forderung nicht bezahlte , l